In [36]:
import os
from pathlib import Path

from scipy.spatial.transform import Rotation as R
import numpy as np
from PIL import Image
from base64 import b64decode
import json

In [48]:
def decode_timestamp(stamp):
    sec = 0
    if "sec" in stamp:
        sec = int(stamp["sec"])
    return sec + float(stamp["nsec"]) / 1e9

def create_image_from_record(d: dict) -> Image:
    raw = b64decode(d["data"])
    pixel_format = d["pixelFormatType"]
    desired_format = "RGB" if pixel_format == "RGB_INT8" else ""
    w = d["width"]
    h = d["height"]
    size = (w, h)
    return Image.frombytes(desired_format, size, raw)

def create_images_from_bag(bag_path: Path, index=0):
    count = 0
    with open(bag_path, "r") as f:
        for l in f.readlines():
            if count < index:
                count += 1
                continue
            d = json.loads(l)
            img = create_image_from_record(d)

            t = decode_timestamp(d["header"]["stamp"])
            sequence = 0
            for i in d["header"]["data"]:
                if i["key"] == "seq":
                    sequence = int(i["value"][0])
                    break
            p = Path(f'{bag_path.parent}/{bag_path.stem}')
            p.mkdir(parents=True, exist_ok=True)
            output_path = p / f'frame_{bag_path.stem}_{t}.jpg'
            img.save(output_path)
            break

def generate_matrix_for_nerfstudio(P: dict, O: dict) -> np.ndarray:
    np.set_printoptions(suppress=True)
    # nerfstudio format
    # [+X0 +Y0 +Z0 X]
    # [+X1 +Y1 +Z1 Y]
    # [+X2 +Y2 +Z2 Z]
    # [0.0 0.0 0.0 1]
    pos = np.array([[P["x"], P["y"], P["z"]]], dtype=float).T
    # pos = pos / 10
    rot = R.from_quat(
        [O["x"], O["y"], O["z"], O["w"]]
        # [0., 0., 0.99999968293183461, 0.00079632671073326324]
        )
    print(rot.as_euler('zyx', degrees=True))
    # print(pos)
    # print(rot)
    pos_rot = np.hstack((rot.as_matrix(), pos))
    # print(pos_rot)
    return np.vstack((pos_rot, np.array([0., 0., 0., 1.], dtype=float)))

def create_transform_matrix(camera_name, database="output/info.bag"):
    with open(database, "r") as f:
        d = json.loads(f.readline())
    for e in d["pose"]:
        if e["name"] != camera_name:
            continue
        pos = {
            "x": 0.,
            "y": 0.,
            "z": 0.,
        }
        pos.update(e["position"])
        orientation = {
            "x": 0.,
            "y": 0.,
            "z": 0.,
            "w": 0.,
        }
        orientation.update(e["orientation"])
        # orientation["w"] -= 1.
        # if orientation["w"] == 0.:
        #     orientation["w"] = 0.00079632671073326324
        # print(e)
        matrix = generate_matrix_for_nerfstudio(pos, orientation)
        print(json.dumps(matrix.tolist(), indent=4))
        break

In [49]:
create_images_from_bag(Path("output/camera1.bag"))

In [47]:
create_transform_matrix("camera1")

[179.90874767   0.          -0.        ]
[
    [
        -0.9999987317275396,
        -0.001592652916486828,
        0.0,
        3.0
    ],
    [
        0.001592652916486828,
        -0.9999987317275396,
        0.0,
        0.0
    ],
    [
        0.0,
        0.0,
        1.0,
        0.55
    ],
    [
        0.0,
        0.0,
        0.0,
        1.0
    ]
]


In [33]:
a = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

In [31]:
a[0:3, 1:3]

array([[2, 3],
       [5, 6],
       [8, 9]])

In [34]:
a[:3, 3]

array([ 4,  8, 12])